# 데이터 전처리

In [53]:
import pandas as pd

train_df = pd.read_csv('./data_base/train.csv')
test_df = pd.read_csv('./data_base/test.csv')

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.head()
# user_id -> 삭제

In [ ]:
train_df.drop('user_id',axis=1,inplace=True)
train_df.head()

### 범주형 문자 데이터 -> Label Encoding

In [ ]:
# dtype_ object -> preferred_difficulty_level	preferred_difficulty_level -> label encoding
train_df['preferred_difficulty_level'].value_counts()


In [ ]:
train_df['preferred_difficulty_level'].value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder

le_sub = LabelEncoder()
le_sub = le_sub.fit(train_df['subscription_type'])

train_df['subscription_type'] = le_sub.transform(train_df['subscription_type'])
train_df['subscription_type'].head()

In [ ]:
le_lev = LabelEncoder()
le_lev = le_lev.fit(train_df['preferred_difficulty_level'])

train_df['preferred_difficulty_level'] = le_lev.transform(train_df['preferred_difficulty_level'])
train_df['preferred_difficulty_level'].head()

In [ ]:
train_df.info()

In [ ]:
train_df

### 이상치 확인

In [ ]:
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots(4,3, figsize = (20,30))

for idx,col in enumerate(train_df.columns[1:13]):
    
    row_index = idx // 3
    col_index = idx % 3

    sns.boxplot(x = train_df['target'],y = train_df[col], color = 'darkseagreen', ax = ax[row_index,col_index])
    ax[row_index,col_index].set_title('boxplot for %s' % col)
plt.show()

plt.tight_layout(pad=5)
plt.subplots_adjust(wspace=0, hspace=5)

#### 이상치 제거 없이 머신러닝 해보기

In [ ]:
# import pandas as pd
# import numpy as np

# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import StratifiedKFold
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# y_train = train_df['target']
# X_train = train_df.drop('target',axis=1)

# train_X,test_X,train_y,test_y = train_test_split(X_train,y_train,
#                                                test_size=0.2,random_state=11)

# dt_clf = DecisionTreeClassifier(random_state=11)
# rf_clf = RandomForestClassifier(random_state=11)
# lr_clf = LogisticRegression(random_state=11)
    
# dt_clf.fit(train_X,train_y)
# rf_clf.fit(train_X,train_y)
# lr_clf.fit(train_X,train_y)
                        
# accuracy_dt = accuracy_score(dt_clf.predict(test_X),test_y)
# accuracy_rf = accuracy_score(rf_clf.predict(test_X),test_y)
# accuracy_lr = accuracy_score(lr_clf.predict(test_X),test_y)


In [ ]:
# # 충격적인 결과
# print(accuracy_dt)
# print(accuracy_rf)
# print(accuracy_lr)

#### 이상치 제거 : IQR

In [ ]:
train_df['customer_inquiry_history'].index

In [ ]:
# outlier 찾기

weight = 1.5
Q1 = train_df['customer_inquiry_history'].quantile(0.25)
Q3 = train_df['customer_inquiry_history'].quantile(0.75)
IQR = Q3-Q1
max_value = Q3 + (IQR * weight)
min_value = Q1 - (IQR * weight)

outlier = []
outlier_index = []
for i in train_df['customer_inquiry_history'].index:
    value = train_df['customer_inquiry_history'][i]

    if value > max_value or value < min_value:
        outlier.append(value)       
        outlier_index.append(i)

In [ ]:
def outlier_feature(dataFrame):
    columns = dataFrame.columns
    outlier = []
    outlier_index = []
    
    for column in columns:
        # 칼럼별 이상치 범위 계산
        weight = 1.5
        Q1 = dataFrame[column].quantile(0.25)
        Q3 = dataFrame[column].quantile(0.75)
        IQR = Q3-Q1
        max_value = Q3 + (IQR * weight)
        min_value = Q1 - (IQR * weight)

        # 칼럼별 이상치 여부 확인 후 인덱스 담기
        for i in train_df[column].index:
            value = train_df[column][i]

            if value > max_value or value < min_value:
                outlier.append(value)       
                outlier_index.append(i)
                
    # 칼럼별 이상치가 있는 행 인덱스 중복 제거
    outlier_index = list(set(outlier_index))
    
    # 이상치 있는 행 제거
    dataFrame.drop(outlier_index, axis=0, inplace=True)
        
    return dataFrame

In [ ]:
train_df.info()

### 제거 확인

In [ ]:
outlier_feature(train_df)
train_df.info()

In [ ]:
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots(4,3, figsize = (20,30))

for idx,col in enumerate(train_df.columns[1:13]):
    
    row_index = idx // 3
    col_index = idx % 3

    sns.boxplot(x = train_df['target'],y = train_df[col], color = 'darkseagreen', ax = ax[row_index,col_index])
    ax[row_index,col_index].set_title('boxplot for %s' % col)
plt.show()

plt.tight_layout(pad=5)
plt.subplots_adjust(wspace=0, hspace=5)

### 전처리 내용 정리
 1. user_id 칼럼 삭제
 2. object 타입 칼럼 라벨인코딩
 3. 이상치 제거 -> IQR

In [54]:

from sklearn import preprocessing

def drop_feature(dataFrame):
    dataFrame.drop('user_id',axis=1,inplace=True)
    return dataFrame
    
def encoding_feature(dataFrame):    
    object_columns = dataFrame.select_dtypes(include='object')
    for column in object_columns.columns:
        le = preprocessing.LabelEncoder()
        le = le.fit(dataFrame[column])
        dataFrame[column] = le.transform(dataFrame[column])
    return dataFrame

def outlier_feature(dataFrame):
    columns = dataFrame.columns
    outlier = []
    outlier_index = []
    
    for column in columns:
        # 칼럼별 이상치 범위 계산
        weight = 1.5
        Q1 = dataFrame[column].quantile(0.25)
        Q3 = dataFrame[column].quantile(0.75)
        IQR = Q3-Q1
        max_value = Q3 + (IQR * weight)
        min_value = Q1 - (IQR * weight)

        # 칼럼별 이상치 여부 확인 후 인덱스 담기
        for i in train_df[column].index:
            value = train_df[column][i]

            if value > max_value or value < min_value:
                outlier.append(value)       
                outlier_index.append(i)
                
    # 칼럼별 이상치가 있는 행 인덱스 중복 제거
    outlier_index = list(set(outlier_index))
    
    # 이상치 있는 행 제거
    dataFrame.drop(outlier_index, axis=0, inplace=True)
        
    return dataFrame
        
        
def preprocessing_df(dataFrame):
    dataFrame = drop_feature(dataFrame)
    dataFrame = encoding_feature(dataFrame)
    dataFrame = outlier_feature(dataFrame)
    return dataFrame

In [55]:
train_df = preprocessing_df(train_df)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9115 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   subscription_duration              9115 non-null   int64  
 1   recent_login_time                  9115 non-null   int64  
 2   average_login_time                 9115 non-null   float64
 3   average_time_per_learning_session  9115 non-null   float64
 4   monthly_active_learning_days       9115 non-null   int64  
 5   total_completed_courses            9115 non-null   int64  
 6   recent_learning_achievement        9115 non-null   float64
 7   abandoned_learning_sessions        9115 non-null   int64  
 8   community_engagement_level         9115 non-null   int64  
 9   preferred_difficulty_level         9115 non-null   int64  
 10  subscription_type                  9115 non-null   int64  
 11  customer_inquiry_history           9115 non-null   int64  
 1

In [56]:
test_df = preprocessing_df(test_df)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   subscription_duration              10000 non-null  int64  
 1   recent_login_time                  10000 non-null  int64  
 2   average_login_time                 10000 non-null  float64
 3   average_time_per_learning_session  10000 non-null  float64
 4   monthly_active_learning_days       10000 non-null  int64  
 5   total_completed_courses            10000 non-null  int64  
 6   recent_learning_achievement        10000 non-null  float64
 7   abandoned_learning_sessions        10000 non-null  int64  
 8   community_engagement_level         10000 non-null  int64  
 9   preferred_difficulty_level         10000 non-null  int64  
 10  subscription_type                  10000 non-null  int64  
 11  customer_inquiry_history           10000 non-null  int6

# 머신러닝

## 기본 머신러닝

In [ ]:
train_df['target'].head(20)

In [58]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

y_train = train_df['target']
X_train = train_df.drop('target',axis=1)

train_X,test_X,train_y,test_y = train_test_split(X_train,y_train,
                                               test_size=0.2,random_state=11)

dt_clf = DecisionTreeClassifier(random_state=11)
rf_clf = RandomForestClassifier(random_state=11)
lr_clf = LogisticRegression(random_state=11)
    
dt_clf.fit(train_X,train_y)
rf_clf.fit(train_X,train_y)
lr_clf.fit(train_X,train_y)
                        
accuracy_dt = accuracy_score(dt_clf.predict(test_X),test_y)
accuracy_rf = accuracy_score(rf_clf.predict(test_X),test_y)
accuracy_lr = accuracy_score(lr_clf.predict(test_X),test_y)


/Users/brielle/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [59]:
print(accuracy_dt)
print(accuracy_rf)
print(accuracy_lr)

0.5117937465715853
0.5655512890839276
0.578167855183763


## 교차검증

In [60]:
from sklearn.model_selection import cross_val_score

CV_scores_dt = cross_val_score(dt_clf,train_X,train_y,scoring='accuracy',cv=5)
CV_scores_rf = cross_val_score(rf_clf,train_X,train_y,scoring='accuracy',cv=5)
CV_scores_lr = cross_val_score(lr_clf,train_X,train_y,scoring='accuracy',cv=5)

/Users/brielle/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/brielle/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

In [61]:
print(np.mean(CV_scores_dt))
print(np.mean(CV_scores_rf))
print(np.mean(CV_scores_lr))

0.5128881705811618
0.5807720115718998
0.6060066133200954


# 추가 전처리

## 피쳐 스케일링

In [85]:
categorical = ['preferred_difficulty_level','subscription_type','target']
uncategorical = train_df.drop(categorical,axis=1)


In [86]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(train_df)
train_scaled = scaler.transform(train_df)
train_df_scaled = pd.DataFrame(data=train_scaled, columns=train_df.columns)

train_df_scaled[categorical] = train_df[categorical]

In [87]:
train_df_scaled.describe()

,subscription_duration,recent_login_time,average_login_time,average_time_per_learning_session,monthly_active_learning_days,total_completed_courses,recent_learning_achievement,abandoned_learning_sessions,community_engagement_level,preferred_difficulty_level,subscription_type,customer_inquiry_history,payment_pattern,target
count,9.115000e+03,9.115000e+03,9.115000e+03,9.115000e+03,9.115000e+03,9.115000e+03,9.115000e+03,9.115000e+03,9.115000e+03,8300.000000,8300.000000,9.115000e+03,9.115000e+03,8300.000000
mean,5.924438e-17,4.677188e-17,6.920289e-16,7.951219e-17,4.521281e-17,-2.408752e-16,6.587039e-16,1.492802e-16,1.262841e-16,1.099759,0.383012,8.574844e-18,1.247250e-17,0.598313
std,1.000055e+00,1.000055e+00,1.000055e+00,1.000055e+00,1.000055e+00,1.000055e+00,1.000055e+00,1.000055e+00,1.000055e+00,0.707439,0.486150,1.000055e+00,1.000055e+00,0.490269
min,-1.646819e+00,-1.676844e+00,-2.735967e+00,-1.178933e+00,-1.672269e+00,-2.593915e+00,-2.737837e+00,-1.808359e+00,-2.249703e+00,0.000000,0.000000,-1.433210e+00,-1.515470e+00,0.000000
25%,-8.904038e-01,-8.410551e-01,-6.739461e-01,-8.100372e-01,-8.052603e-01,-6.080169e-01,-6.967423e-01,-5.918871e-01,-6.781178e-01,1.000000,0.000000,-7.108597e-01,-1.083014e+00,0.000000
50%,1.729423e-02,-5.265845e-03,-5.818882e-03,-2.822042e-01,6.174798e-02,-4.061746e-02,4.971651e-03,1.634863e-02,1.076747e-01,1.000000,0.000000,1.149104e-02,2.143539e-01,1.000000
75%,9.249923e-01,8.305234e-01,6.813112e-01,5.584841e-01,9.287562e-01,8.104817e-01,6.840769e-01,6.245844e-01,8.934672e-01,2.000000,1.000000,7.338418e-01,1.079266e+00,1.000000
max,1.681407e+00,1.666313e+00,2.741840e+00,3.075431e+00,1.651263e+00,2.796380e+00,2.749703e+00,2.449292e+00,8.934672e-01,2.000000,1.000000,2.900894e+00,1.511722e+00,1.000000


In [88]:
train_df_scaled.fillna(0,inplace=True)

In [89]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

y_train = train_df_scaled['target']
X_train = train_df_scaled.drop('target',axis=1)

train_X,test_X,train_y,test_y = train_test_split(X_train,y_train,
                                               test_size=0.2,random_state=11)

# print(train_X)
# print(test_X)
# print(train_y)
# print(test_y)



dt_clf = DecisionTreeClassifier(random_state=11)
rf_clf = RandomForestClassifier(random_state=11)
lr_clf = LogisticRegression(random_state=11)
    
dt_clf.fit(train_X,train_y)
rf_clf.fit(train_X,train_y)
lr_clf.fit(train_X,train_y)
                        
accuracy_dt = accuracy_score(dt_clf.predict(test_X),test_y)
accuracy_rf = accuracy_score(rf_clf.predict(test_X),test_y)
accuracy_lr = accuracy_score(lr_clf.predict(test_X),test_y)


In [90]:
print(accuracy_dt)
print(accuracy_rf)
print(accuracy_lr)

0.5095995611629183
0.5792649478880966
0.5929786066922655


# 하이퍼 파라미터 튜닝

* 의사결정나무

In [62]:
from sklearn.model_selection import GridSearchCV

parameters = {'max_depth':[2,5,10],'min_samples_leaf':[2,5,10]}
grid_dt_clf = GridSearchCV(dt_clf,param_grid = parameters,cv=5, refit=True)
grid_dt_clf.fit(X_train,y_train)

scores_df_dt = pd.DataFrame(grid_dt_clf.cv_results_)


In [63]:
scores_df_dt[['params','mean_test_score','rank_test_score']]

,params,mean_test_score,rank_test_score
0,"{'max_depth': 2, 'min_samples_leaf': 2}",0.600549,1
1,"{'max_depth': 2, 'min_samples_leaf': 5}",0.600549,1
2,"{'max_depth': 2, 'min_samples_leaf': 10}",0.600549,1
3,"{'max_depth': 5, 'min_samples_leaf': 2}",0.598354,4
4,"{'max_depth': 5, 'min_samples_leaf': 5}",0.598025,5
5,"{'max_depth': 5, 'min_samples_leaf': 10}",0.597257,6
6,"{'max_depth': 10, 'min_samples_leaf': 2}",0.580143,9
7,"{'max_depth': 10, 'min_samples_leaf': 5}",0.580911,7
8,"{'max_depth': 10, 'min_samples_leaf': 10}",0.580911,8


In [69]:
print('GridSearchCV 최적 하이퍼 파라미터: ',grid_dt_clf.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(grid_dt_clf.best_score_))
best_dt_clf = grid_dt_clf.best_estimator_

GridSearchCV 최적 하이퍼 파라미터:  {'max_depth': 2, 'min_samples_leaf': 2}
GridSearchCV 최고 정확도: 0.6005


In [72]:
# GridSearchCV의 최적 하이퍼 파라미터로 학습된 Estimator로 예측 및 평가 수행

best_dt_pred = best_dt_clf.predict(test_X)

accuracy = accuracy_score(test_y,best_dt_pred)

print('테스트 세트에서의 DecisionTreeClassifier 정확도: {0:.4f}'.format(accuracy))

테스트 세트에서의 DecisionTreeClassifier 정확도: 0.5782


In [73]:
# 교차검증
best_dt_clf_score = cross_val_score(best_dt_clf,train_X,train_y,scoring='accuracy',cv=5)
print(np.round(np.mean(best_dt_clf_score),4))

0.6059


* 로지스틱회귀

In [92]:
from sklearn.model_selection import GridSearchCV

params = {'penalty': ['l2'], 'C': [0.01, 0.1, 1, 5, 10]}
grid_lr_clf = GridSearchCV(lr_clf, param_grid=params, cv=5, refit=True)
grid_lr_clf.fit(X_train,y_train)

scores_df_lr = pd.DataFrame(grid_lr_clf.cv_results_)


In [93]:
scores_df_lr[['params','mean_test_score','rank_test_score']]

,params,mean_test_score,rank_test_score
0,"{'C': 0.01, 'penalty': 'l2'}",0.595283,1
1,"{'C': 0.1, 'penalty': 'l2'}",0.591991,2
2,"{'C': 1, 'penalty': 'l2'}",0.591772,3
3,"{'C': 5, 'penalty': 'l2'}",0.591662,4
4,"{'C': 10, 'penalty': 'l2'}",0.591662,4


# to do
* 추가 전처리
1. feature들 중에 연관있는 애들끼리 묶어보기 (학습시간 / 커뮤니티 참여 등)
2. 